# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name:

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [17]:
import requests
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import xmltodict


# Use a song that works with the ChartLyrics API
artist = "Coldplay"
song = "Yellow"

# API endpoint
url = f"http://api.chartlyrics.com/apiv1.asmx/SearchLyricDirect?artist={artist}&song={song}"

# Fetch and parse
response = requests.get(url)

if response.status_code == 200:
    data = xmltodict.parse(response.text)
    lyrics = data.get("GetLyricResult", {}).get("Lyric")

    if lyrics and lyrics.strip():
        result = {
            "artist": artist,
            "song": song,
            "lyrics": lyrics
        }
        with open("yellow_lyrics.json", "w", encoding='utf-8') as file:
            json.dump(result, file, indent=4)
        print("✅ Lyrics saved to yellow_lyrics.json")
    else:
        print("❌ No lyrics found in the API response.")
else:
    print(f"❌ Request failed: {response.status_code}")
    print("Response:", repr(response.text))




✅ Lyrics saved to yellow_lyrics.json


2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [24]:
import json
import spacy
from spacy.tokens import Doc
from textblob import TextBlob


# ✅ Register extensions (if not already registered)
if not Doc.has_extension('blob'):
    Doc.set_extension('blob', getter=lambda doc: TextBlob(doc.text))
if not Doc.has_extension('polarity'):
    Doc.set_extension('polarity', getter=lambda doc: doc._.blob.sentiment.polarity)

# ✅ Load spaCy model
nlp = spacy.load("en_core_web_sm")

# ✅ Load JSON file (replace with your saved filename)
with open('yellow_lyrics.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# ✅ Extract lyrics from JSON
lyrics = data.get('lyrics', '')

if lyrics.strip():
    # ✅ Process lyrics with spaCy
    doc = nlp(lyrics)

    # ✅ Print sentiment scores
    print(f"Analyzing: '{data.get('song')}' by {data.get('artist')}")
    print("Polarity:", doc._.polarity)
    print("\nLyrics Preview:\n", lyrics[:500])
else:
    print("⚠️ No lyrics found in the file.")


Analyzing: 'Yellow' by Coldplay
Polarity: 0.2430555555555556

Lyrics Preview:
 Look at the stars
Look how they shine for you
And everything you do
Yeah, they were all yellow

I came along
I wrote a song for you
And all the things you do
And it was called Yellow

So then I took my turn
Oh what a thing to've done
And it was all yellow

Your skin
Oh yeah your skin and bones
Turn it into something beautiful
And you know
You know I love you so
You know I love you so

I swam across
I jumped across for you
Oh what a thing to do
'Cause you were all yellow

I drew a line
I drew a l


3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [28]:
def get_chartlyrics_lyrics(artist, song, filename):
    """
    Fetches lyrics using the ChartLyrics XML API, converts to JSON, and saves to a file.
    """
    url = f"http://api.chartlyrics.com/apiv1.asmx/SearchLyricDirect?artist={artist}&song={song}"
    response = requests.get(url)

    if response.status_code == 200:
        data_dict = xmltodict.parse(response.text)
        lyrics = data_dict.get("GetLyricResult", {}).get("Lyric", "")

        if lyrics and lyrics.strip():
            result = {
                "artist": artist,
                "song": song,
                "lyrics": lyrics
            }
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(result, f, indent=4)
            print(f"Saved lyrics for {artist} - {song} to {filename}")
        else:
            print(f"No lyrics found for {artist} - {song}")
    else:
        print(f"Request failed for {artist} - {song}. Status: {response.status_code}")


# ✅ WORKING SONGS

get_chartlyrics_lyrics("Bon Jovi", "It's My Life", "bon_jovi_its_my_life.json")
get_chartlyrics_lyrics("Linkin Park", "Numb", "linkin_park_numb.json")
get_chartlyrics_lyrics("Green Day", "Boulevard of Broken Dreams", "green_day_boulevard.json")


Saved lyrics for Bon Jovi - It's My Life to bon_jovi_its_my_life.json
Saved lyrics for Linkin Park - Numb to linkin_park_numb.json
Saved lyrics for Green Day - Boulevard of Broken Dreams to green_day_boulevard.json


4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [30]:
def analyze_lyrics_polarity(filename):
    """
    Loads a lyrics JSON file, runs sentiment analysis,
    and returns the polarity score.
    """
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    lyrics = data.get("lyrics", "")
    song_title = data.get("song", "Unknown")

    if not lyrics.strip():
        print(f"⚠️ No lyrics found in {filename}")
        return song_title, None

    doc = nlp(lyrics)
    return song_title, doc._.polarity

# ✅ Analyze 3 songs
files = [
    "green_day_boulevard.json",
    "bon_jovi_its_my_life.json",
    "linkin_park_numb.json"
]

for file in files:
    song, polarity = analyze_lyrics_polarity(file)
    if polarity is not None:
        print(f"{song}: Polarity Score = {polarity:.3f}")
    else:
        print(f"Could not analyze {song}")

Boulevard of Broken Dreams: Polarity Score = -0.126
It's My Life: Polarity Score = 0.109
Numb: Polarity Score = -0.049
